In [4]:
import json
import os
import pandas as pd
import numpy as np

data_path = "UNI/TFG/code/Million Playlist Dataset/spotify_million_playlist_dataset/data/"

# List with all mpd data files path
files = sorted(os.listdir(data_path))

In [ ]:
playlists = {}
for f in files:
    print(f)
    mpd = open(data_path + f)
    mpd = json.load(mpd)['playlists']
    
    for playlist in mpd:
        pid = playlist['pid']
        playlists[pid] = []
        tracks = playlist['tracks']
        for track in tracks:
            playlists[pid].append(track['track_name'])

In [21]:
from collections import Iterable

def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:        
            yield item

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


In [27]:
songs = list(flatten(playlists.values()))
songs_test = list(set(songs))

In [32]:
song_to_id_dict = {}

sid = 0
for song in songs_test:
    song_to_id_dict[song] = sid
    sid += 1

In [36]:
playlist_to_sid = {}

for pid in playlists.keys():
    playlist_to_sid[pid] = []
    for song in playlists[pid]:
        playlist_to_sid[pid].append(song_to_id_dict[song])

In [47]:
import scipy.sparse as sp

mat = sp.dok_matrix((len(playlist_to_sid.keys()),len(songs_test)), dtype=np.int8)

for playlist, song in playlist_to_sid.items():
    mat[playlist, song] = 1
    
mat = mat.tocsr()

In [53]:
sp.save_npz("sparsemat/mat.npz", mat)

In [55]:
f = open("sparsemat/playlist_to_sid.txt","w")

# write file
f.write( str(playlist_to_sid) )

# close file
f.close()

In [56]:
f = open("sparsemat/song_to_id_dict.txt","w")

# write file
f.write( str(song_to_id_dict) )

# close file
f.close()